In [3]:
import math
import torch
import torch.nn as nn

from models.classifier import Classifier
from pytorch_pretrained_bert import BertModel, BertConfig

In [ ]:
#Hyperparameter
num_hiddens = 16
vocab

In [2]:
class Bert(nn.Module):
    def __init__(self, load_pretrained_bert, bert_config = None, tmp_dir='./dataset/bert/', ):
        super(Bert, self).__init__()
        if load_pretrained_bert:
            self.model = BertModel.from_pretrained('bert-base-uncased', cache_dir=tmp_dir)
        else:
            assert bert_config, 'Bert config cannot be null, or set load_pretrained_bert to True'
            self.model = BertModel(bert_config)
        
    
    def forward(self, x, segs, mask):
        #attention mask is like valid_lens
        encoded_layers, _ = self.model(x, segs, attention_mask = mask)
        top_vec = encoded_layers[-1]
        return top_vec

In [ ]:
class Summarizer(nn.Module):
    def __init__(self, args, device, load_pretrained_bert = False, bert_config = None):
        super(Summarizer, self).__init__()

        #Setup baseline
        bert_config = BertConfig(
            args.vovcab_size,
            hidden_size = args.hidden_size,
            num_hidden_layers = args.num_hidden_layers,
            num_attention_heads = args.num_attention_heads,
            intermediate_size=args.rnn_size
        )
        self.bert = Bert(False, bert_config)
        self.encoder = Classifier(self.bert.model.config.num_hiddens)

        #init arguments
        if args.param_init != 0.0:
            for p in self.encoder.parameters():
                p.data.uniform_(-args.param_init, args.param_init)
        
        self.to(device)
    
    def forward(self):
        pass
    